In [1]:
import pandas as pd

from Model_Code.iohmm_nn_model import IOHMM_NN
from Model_Code.rnn_model import RNN
from misc_fns import *
                                                                                           
import torch

In [2]:
iohmm = torch.load("Model/iohmm_nn_large.pth", weights_only=False)
rnn   = torch.load("Model/rnn_large.pth",      weights_only=False)

In [3]:
path = "Data/GOOGL.csv"
df = pd.read_csv(path)
df['Date'] = pd.to_datetime(df['Date'])

cutoff_date = pd.Timestamp('2020-01-01')
train_df = df[df['Date'] < cutoff_date]
test_df = df[df['Date'] >= cutoff_date]

In [4]:
inputs_train, targets_train = get_dataset(train_df, window_size=3)
inputs_test,  targets_test  = get_dataset(test_df,  window_size=3)

In [5]:
rnn_train_preformance   = performance(rnn,   inputs_train, targets_train, torch.nn.MSELoss())
rnn_test_performance    = performance(rnn,   inputs_test,  targets_test,  torch.nn.MSELoss())

iohmm_train_performance = performance(iohmm, inputs_train, targets_train, torch.nn.MSELoss())
iohmm_test_performance  = performance(iohmm, inputs_test,  targets_test,  torch.nn.MSELoss())

In [6]:
import pandas as pd

# Extract loss and r_square values from the performance dictionaries
loss_data = {
    'train': [iohmm_train_performance['loss'], rnn_train_preformance['loss']],
    'test': [iohmm_test_performance['loss'], rnn_test_performance['loss']]
}

r_square_data = {
    'train': [iohmm_train_performance['r_square'], rnn_train_preformance['r_square']],
    'test': [iohmm_test_performance['r_square'], rnn_test_performance['r_square']]
}

# Create punnet squares as DataFrames
loss_df = pd.DataFrame(loss_data, index=['iohmm', 'rnn']).transpose()
r_square_df = pd.DataFrame(r_square_data, index=['iohmm', 'rnn']).transpose() * 100

In [7]:
loss_df

,iohmm,rnn
train,433.260529,80.672798
test,7770.721191,1374.044189


In [8]:
r_square_df

,iohmm,rnn
train,99.630970,99.931288
test,97.779745,99.607408
